Primary inspirations:

- https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [40]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import numpy as np

In [2]:
# Install nltk things
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kenneth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kenneth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Get document-topic matrix
def get_document_topic_matrix(model, bow):
    x = []
    for d in bow:
        # Get document's topics
        doc_topics = lda.get_document_topics(d)
        
        # Extract just the document-topic probabilities
        x.append([i[1] for i in doc_topics])

    return x

# Case 1: 3 docs, 1 topic per doc

Expect to see three topics, with the majority of the probability weights to correspond to the known topic

In [4]:
# Create "docs"
docs = [
    'a1 a2 a3',
    'b1 b2 b3',
    'c1 c2 c3',    
]

# Split by space
docs = [d.split() for d in docs]

# Inspect
print(docs)

[['a1', 'a2', 'a3'], ['b1', 'b2', 'b3'], ['c1', 'c2', 'c3']]


In [5]:
# Create dictionary
dic = corpora.Dictionary(docs)

# Create bow matrix
bow = [dic.doc2bow(doc) for doc in docs]

# Inspect
bow

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1)]]

In [6]:
# Train model
lda = gensim.models.ldamodel.LdaModel(
    bow,
    num_topics=3,
    id2word = dic,
    passes=50,
    random_state=666
)

In [7]:
# Inspect topics
lda.show_topics()

[(0,
  '0.222*"a1" + 0.222*"a2" + 0.222*"a3" + 0.056*"b1" + 0.056*"b2" + 0.056*"b3" + 0.056*"c1" + 0.056*"c3" + 0.056*"c2"'),
 (1,
  '0.222*"c1" + 0.222*"c2" + 0.222*"c3" + 0.056*"a3" + 0.056*"b2" + 0.056*"b3" + 0.056*"a1" + 0.056*"a2" + 0.056*"b1"'),
 (2,
  '0.222*"b1" + 0.222*"b2" + 0.222*"b3" + 0.056*"a2" + 0.056*"a3" + 0.056*"c3" + 0.056*"a1" + 0.056*"c1" + 0.056*"c2"')]

In [8]:
# Document-topic matrix
np.round(get_document_topic_matrix(lda, bow), 2)

array([[0.83, 0.08, 0.08],
       [0.08, 0.08, 0.83],
       [0.08, 0.83, 0.08]], dtype=float32)

In [9]:
# Topic-term matrix
np.round(lda.get_topics(), 2)

array([[0.22, 0.22, 0.22, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06],
       [0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.22, 0.22, 0.22],
       [0.06, 0.06, 0.06, 0.22, 0.22, 0.22, 0.06, 0.06, 0.06]],
      dtype=float32)

# Case 1.1: Many docs

A little surprised that the off-topic words have such a high relative probability of appearing the topic (~6% for each word that doesn't appear at all in the docs with the topics).  But this might be primarily a sample size thing.

Repeat Case 1 but instead have ~1k randomly sampled docs from the three used in Case 1

In [10]:
# Create "docs"
np.random.seed(666)

docs_sample = np.random.choice(['a', 'b', 'c'], size=100)

docs = [f'{x}1 {x}2 {x}3' for x in docs_sample]

# Split by space
docs = [d.split() for d in docs]

# Inspect
print(docs[:10])

[['a1', 'a2', 'a3'], ['c1', 'c2', 'c3'], ['b1', 'b2', 'b3'], ['c1', 'c2', 'c3'], ['c1', 'c2', 'c3'], ['c1', 'c2', 'c3'], ['b1', 'b2', 'b3'], ['c1', 'c2', 'c3'], ['a1', 'a2', 'a3'], ['b1', 'b2', 'b3']]


In [11]:
# Create dictionary
dic = corpora.Dictionary(docs)

# Create bow matrix
bow = [dic.doc2bow(doc) for doc in docs]

# Inspect
bow[:10]

[[(0, 1), (1, 1), (2, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1)],
 [(3, 1), (4, 1), (5, 1)],
 [(0, 1), (1, 1), (2, 1)],
 [(6, 1), (7, 1), (8, 1)]]

In [12]:
# Train model
lda = gensim.models.ldamodel.LdaModel(
    bow,
    num_topics = 3,
    id2word = dic,
    passes=50,
    random_state=666
)

In [13]:
# Inspect topics
lda.show_topics()

[(0,
  '0.327*"a1" + 0.327*"a2" + 0.327*"a3" + 0.003*"c1" + 0.003*"c2" + 0.003*"c3" + 0.003*"b3" + 0.003*"b1" + 0.003*"b2"'),
 (1,
  '0.326*"b1" + 0.326*"b2" + 0.326*"b3" + 0.004*"a3" + 0.004*"a1" + 0.004*"a2" + 0.004*"c3" + 0.004*"c2" + 0.004*"c1"'),
 (2,
  '0.327*"c1" + 0.327*"c2" + 0.327*"c3" + 0.003*"a3" + 0.003*"a2" + 0.003*"a1" + 0.003*"b3" + 0.003*"b1" + 0.003*"b2"')]

In [14]:
# Document-topic matrix
np.round(get_document_topic_matrix(lda, bow), 2)[:5, :]

array([[0.83, 0.08, 0.08],
       [0.08, 0.08, 0.83],
       [0.08, 0.83, 0.08],
       [0.08, 0.08, 0.83],
       [0.08, 0.08, 0.83]], dtype=float32)

In [15]:
# Topic-term matrix
np.round(lda.get_topics(), 2)

array([[0.33, 0.33, 0.33, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.33, 0.33, 0.33],
       [0.  , 0.  , 0.  , 0.33, 0.33, 0.33, 0.  , 0.  , 0.  ]],
      dtype=float32)

# Case 2: 2 equal topics per doc

In [16]:
# Create "docs"
docs = [
    'a1 a2 a3 b1 b2 b3',
    'b1 b2 b3 c1 c2 c3',
    'c1 c2 c3 a1 a2 a3',    
]

# Split by space
docs = [d.split() for d in docs]

# Inspect
print(docs)

[['a1', 'a2', 'a3', 'b1', 'b2', 'b3'], ['b1', 'b2', 'b3', 'c1', 'c2', 'c3'], ['c1', 'c2', 'c3', 'a1', 'a2', 'a3']]


In [17]:
# Create dictionary
dic = corpora.Dictionary(docs)

# Create bow matrix
bow = [dic.doc2bow(doc) for doc in docs]

# Inspect
bow

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1)]]

In [18]:
# Train model
lda = gensim.models.ldamodel.LdaModel(
    bow,
    num_topics=3,
    id2word = dic,
    passes=50,
    random_state=666
)

In [19]:
# Inspect topics
lda.show_topics()

[(0,
  '0.192*"a2" + 0.192*"a1" + 0.192*"a3" + 0.111*"b1" + 0.111*"b2" + 0.111*"b3" + 0.030*"c3" + 0.030*"c1" + 0.030*"c2"'),
 (1,
  '0.192*"c2" + 0.192*"c1" + 0.192*"c3" + 0.111*"b2" + 0.111*"b3" + 0.111*"b1" + 0.030*"a3" + 0.030*"a1" + 0.030*"a2"'),
 (2,
  '0.111*"b3" + 0.111*"b2" + 0.111*"b1" + 0.111*"a3" + 0.111*"a2" + 0.111*"a1" + 0.111*"c3" + 0.111*"c1" + 0.111*"c2"')]

In [20]:
# Topic-term matrix
np.round(lda.get_topics(), 2)

array([[0.19, 0.19, 0.19, 0.11, 0.11, 0.11, 0.03, 0.03, 0.03],
       [0.03, 0.03, 0.03, 0.11, 0.11, 0.11, 0.19, 0.19, 0.19],
       [0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11]],
      dtype=float32)

In [21]:
# Document-topic matrix
np.round(get_document_topic_matrix(lda, bow), 2)

array([[0.9 , 0.05, 0.05],
       [0.05, 0.9 , 0.05],
       [0.48, 0.48, 0.05]], dtype=float32)

# Case 2.1: Many docs

A little surprised that one of the topics has "everything" as an important element.  But this might also be primarily a sample size thing.

Repeat Case 2 but instead have ~1k randomly sampled docs from the three used in Case 2

In [22]:
# Create "docs"
np.random.seed(1337)

docs_sample = np.random.choice(['a', 'b', 'c'], size=100)

def gen_string(x):
    if x == 'a':
        return 'a1 a2 a3 b1 b2 b3'
    elif x == 'b':
        return 'b1 b2 b3 c1 c2 c3'
    elif x == 'c':
        return 'c1 c2 c3 a1 a2 a3'
    raise ValueError(f'value {x} not supported')

docs = [gen_string(x) for x in docs_sample]

# Split by space
docs = [d.split() for d in docs]

# Inspect
print(docs[:10])

[['b1', 'b2', 'b3', 'c1', 'c2', 'c3'], ['a1', 'a2', 'a3', 'b1', 'b2', 'b3'], ['a1', 'a2', 'a3', 'b1', 'b2', 'b3'], ['b1', 'b2', 'b3', 'c1', 'c2', 'c3'], ['c1', 'c2', 'c3', 'a1', 'a2', 'a3'], ['c1', 'c2', 'c3', 'a1', 'a2', 'a3'], ['c1', 'c2', 'c3', 'a1', 'a2', 'a3'], ['a1', 'a2', 'a3', 'b1', 'b2', 'b3'], ['a1', 'a2', 'a3', 'b1', 'b2', 'b3'], ['b1', 'b2', 'b3', 'c1', 'c2', 'c3']]


In [23]:
# Create dictionary
dic = corpora.Dictionary(docs)

# Create bow matrix
bow = [dic.doc2bow(doc) for doc in docs]

# Inspect
bow[:5]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1)],
 [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]

In [24]:
# Train model
lda = gensim.models.ldamodel.LdaModel(
    bow,
    num_topics=3,
    id2word = dic,
    passes=50,
    random_state=666
)

In [25]:
# Inspect topics
lda.show_topics()

[(0,
  '0.166*"a1" + 0.166*"a2" + 0.166*"a3" + 0.166*"b1" + 0.166*"b2" + 0.166*"b3" + 0.002*"c1" + 0.002*"c2" + 0.002*"c3"'),
 (1,
  '0.166*"a2" + 0.166*"a1" + 0.166*"a3" + 0.166*"c2" + 0.166*"c3" + 0.166*"c1" + 0.002*"b3" + 0.002*"b1" + 0.002*"b2"'),
 (2,
  '0.166*"b2" + 0.166*"b3" + 0.166*"b1" + 0.166*"c1" + 0.166*"c2" + 0.166*"c3" + 0.002*"a3" + 0.002*"a2" + 0.002*"a1"')]

In [26]:
# Topic-term matrix
np.round(lda.get_topics(), 2)

array([[0.17, 0.17, 0.17, 0.  , 0.  , 0.  , 0.17, 0.17, 0.17],
       [0.  , 0.  , 0.  , 0.17, 0.17, 0.17, 0.17, 0.17, 0.17],
       [0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.  , 0.  , 0.  ]],
      dtype=float32)

In [27]:
# Document-topic matrix
np.round(get_document_topic_matrix(lda, bow), 2)[:5, :]

array([[0.05, 0.05, 0.9 ],
       [0.9 , 0.05, 0.05],
       [0.9 , 0.05, 0.05],
       [0.05, 0.05, 0.9 ],
       [0.05, 0.9 , 0.05]], dtype=float32)

Much better, though interesting that we're still seeing some "off topic" percentages of generally ~10% per doc

# Case 3: 3 cascading topics per doc

In [28]:
# Create "docs"
docs = [
    'a1 a2 a3 b1 b2 c1',
    'b1 b2 b3 c1 c2 a1',
    'c1 c2 c3 a1 a2 b1',    
]

# Split by space
docs = [d.split() for d in docs]

# Inspect
print(docs)

[['a1', 'a2', 'a3', 'b1', 'b2', 'c1'], ['b1', 'b2', 'b3', 'c1', 'c2', 'a1'], ['c1', 'c2', 'c3', 'a1', 'a2', 'b1']]


In [29]:
# Create dictionary
dic = corpora.Dictionary(docs)

# Create bow matrix
bow = [dic.doc2bow(doc) for doc in docs]

# Inspect
bow

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (1, 1), (3, 1), (5, 1), (7, 1), (8, 1)]]

In [30]:
# Train model
lda = gensim.models.ldamodel.LdaModel(
    bow,
    num_topics=3,
    id2word = dic,
    passes=50,
    random_state=666
)

In [31]:
# Inspect topics
lda.show_topics()

[(0,
  '0.148*"a1" + 0.148*"b1" + 0.148*"c1" + 0.148*"a2" + 0.148*"b2" + 0.148*"a3" + 0.037*"c2" + 0.037*"c3" + 0.037*"b3"'),
 (1,
  '0.148*"c1" + 0.148*"a1" + 0.148*"b1" + 0.148*"c2" + 0.148*"b2" + 0.148*"b3" + 0.037*"a2" + 0.037*"a3" + 0.037*"c3"'),
 (2,
  '0.148*"c1" + 0.148*"b1" + 0.148*"a1" + 0.148*"c2" + 0.148*"a2" + 0.148*"c3" + 0.037*"b2" + 0.037*"a3" + 0.037*"b3"')]

In [32]:
# Topic-term matrix
np.round(lda.get_topics(), 2)

array([[0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.04, 0.04, 0.04],
       [0.15, 0.04, 0.04, 0.15, 0.15, 0.15, 0.15, 0.15, 0.04],
       [0.15, 0.15, 0.04, 0.15, 0.04, 0.15, 0.04, 0.15, 0.15]],
      dtype=float32)

In [33]:
# Document-topic matrix
np.round(get_document_topic_matrix(lda, bow), 2)

array([[0.89, 0.05, 0.05],
       [0.05, 0.89, 0.05],
       [0.05, 0.05, 0.89]], dtype=float32)

# Case 2.1: Many docs

Directionally isn't bad, but yet again it's surprising it's not distinguishing between primary and secondary topics.  But this might also be primarily a sample size thing.

Repeat Case 3 but instead have ~1k randomly sampled docs from the three used in Case 3

In [34]:
# Create "docs"
np.random.seed(369)

docs_sample = np.random.choice(['a', 'b', 'c'], size=100)

def gen_string(x):
    if x == 'a':
        return 'a1 a2 a3 b1 b2 c1'
    elif x == 'b':
        return 'b1 b2 b3 c1 c2 a1'
    elif x == 'c':
        return 'c1 c2 c3 a1 a2 b1'
    raise ValueError(f'value {x} not supported')

docs = [gen_string(x) for x in docs_sample]

# Split by space
docs = [d.split() for d in docs]

# Inspect
print(docs[:10])

[['c1', 'c2', 'c3', 'a1', 'a2', 'b1'], ['b1', 'b2', 'b3', 'c1', 'c2', 'a1'], ['c1', 'c2', 'c3', 'a1', 'a2', 'b1'], ['c1', 'c2', 'c3', 'a1', 'a2', 'b1'], ['a1', 'a2', 'a3', 'b1', 'b2', 'c1'], ['a1', 'a2', 'a3', 'b1', 'b2', 'c1'], ['c1', 'c2', 'c3', 'a1', 'a2', 'b1'], ['c1', 'c2', 'c3', 'a1', 'a2', 'b1'], ['c1', 'c2', 'c3', 'a1', 'a2', 'b1'], ['c1', 'c2', 'c3', 'a1', 'a2', 'b1']]


In [35]:
# Create dictionary
dic = corpora.Dictionary(docs)

# Create bow matrix
bow = [dic.doc2bow(doc) for doc in docs]

# Inspect
bow[:5]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(0, 1), (2, 1), (3, 1), (4, 1), (6, 1), (7, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (6, 1), (8, 1)]]

In [36]:
# Train model
lda = gensim.models.ldamodel.LdaModel(
    bow,
    num_topics=3,
    id2word = dic,
    passes=50,
    random_state=666
)

In [37]:
# Inspect topics
lda.show_topics()

[(0,
  '0.166*"a2" + 0.166*"a3" + 0.166*"a1" + 0.166*"c1" + 0.166*"b1" + 0.166*"b2" + 0.002*"c2" + 0.002*"c3" + 0.002*"b3"'),
 (1,
  '0.166*"b1" + 0.166*"a1" + 0.166*"c1" + 0.165*"c2" + 0.165*"b2" + 0.165*"b3" + 0.002*"a2" + 0.002*"c3" + 0.002*"a3"'),
 (2,
  '0.166*"c3" + 0.166*"a2" + 0.166*"c2" + 0.166*"b1" + 0.166*"c1" + 0.166*"a1" + 0.001*"b2" + 0.001*"a3" + 0.001*"b3"')]

In [38]:
# Topic-term matrix
np.round(lda.get_topics(), 2)

array([[0.17, 0.17, 0.17, 0.17, 0.  , 0.  , 0.17, 0.  , 0.17],
       [0.17, 0.  , 0.17, 0.17, 0.17, 0.  , 0.17, 0.16, 0.  ],
       [0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.  , 0.  , 0.  ]],
      dtype=float32)

In [39]:
# Document-topic matrix
np.round(get_document_topic_matrix(lda, bow), 2)[:5, :]

array([[0.05, 0.05, 0.89],
       [0.05, 0.89, 0.05],
       [0.05, 0.05, 0.89],
       [0.05, 0.05, 0.89],
       [0.89, 0.05, 0.05]], dtype=float32)

In [ ]:
# Optimal number of topics